# Importing and Reading the data

In [ ]:
#import libraries
from sklearn.datasets import load_files
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import RandomizedSearchCV 
import matplotlib.pyplot as plt
import numpy as np
import datetime
from datetime import date, timedelta
import pandas as pd
import seaborn as sns
from google.colab import drive
! pip install pattern
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import pattern
from pattern.en import lemma, lexeme

In [ ]:
# pick data from google drive 
drive.mount('/content/gdrive')
path = "/content/gdrive/MyDrive/Noticias_Premier_League/"
path2= "/content/gdrive/MyDrive/finaldataset.csv"

#path = '/content/gdrive/MyDrive/Noticias_Premier_League'
#path2 = '/content/gdrive/MyDrive/csv_files/finaldataset.csv'



Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


# Preparing Data

## Create functions to use in DataFrame


In [ ]:
# Create function to change de format of date
def change_data(v,x,d):
  if len(v[d][x])==8:
    year = '2'+'0'+v[d][x][6]+v[d][x][7]
    month = v[d][x][3]+ v[d][x][4]
    day = v[d][x][0]+ v[d][x][1]
    date = year+'-'+month+'-'+day
    return date
  elif len(v[d][x])==10:
      year = v[d][x][6]+v[d][x][7]+v[d][x][8]+v[d][x][9]
      month = v[d][x][3]+ v[d][x][4]
      day = v[d][x][0]+ v[d][x][1]
      date = year+'-'+month+'-'+day
      return date


In [ ]:
# Create function to apply dummies
def dummies(resultado_d,x):
  if x == 'H':
    resultado_d.append(1)
  elif x=='D':
    resultado_d.append(2)
  elif x=='A':
    resultado_d.append(3)
    

In [ ]:
# Create a function to pick news ten days ago the game 
def get_news(dataframe,x,timev,casa):
  time = dataframe[casa][x] 
  vetor_resultado = []
  if time in timev:
    vetor_noticias=[]
    arquivo_noticias = times[time]
    data_atual = date.fromisoformat(dataframe['Date'][x])
    for y in range (1,11):
      data_analisada = data_atual - timedelta(days=y)
      data = data_analisada.isoformat()
      n_atual =arquivo_noticias.loc[arquivo_noticias['date']==data]
      vetor = np.array(n_atual)
      for k in range (0,len(vetor)):
          texto = ''
          for h in range(3,6):
            texto = texto+''+vetor[k][h]
          vetor_noticias.append(texto)
    data_frame_vetor = pd.DataFrame(vetor_noticias)
    data_frame_vetor.drop_duplicates(inplace = True)
    vetor_resultado = np.array(data_frame_vetor)
  return vetor_resultado

##Create dataframes using created fuctions in data


In [ ]:
## tirar esse unnamed (quando ler o df, fazer index_col = [0])

In [ ]:
# Create dataframe with date of games and replace abbreviations of teams
dfp = pd.read_csv(path2, index_col = [0])
dfp.replace('Man United','Manchester United',inplace=True)
dfp.replace ('Man City','Manchester City',inplace=True)
dfp


,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HS,AS,HST,AST,HC,AC,HF,AF,HY,AY,HR,AR,HTGS,ATGS,HTGC,ATGC,HTP,ATP,HM1,AM1,HM2,AM2,HM3,AM3,HM4,AM4,HM5,AM5,HomeTeamLP,AwayTeamLP,MW,HTFormPtsStr,ATFormPtsStr,HTFormPts,ATFormPts,HTGD,ATGD,DiffPts,DiffFormPts,DiffLP
0,19/08/00,Charlton,Manchester City,4,0,H,17,8,14,4,6,6,13,12,1,2,0,0,0,0,0,0,0.000000,0.000000,M,M,M,M,M,M,M,M,M,M,0.0,0.0,1.0,MMMMM,MMMMM,0,0,0.000000,0.000000,0.000000,0.000000,0.0
1,19/08/00,Chelsea,West Ham,4,2,H,17,12,10,5,7,7,19,14,1,2,0,0,0,0,0,0,0.000000,0.000000,M,M,M,M,M,M,M,M,M,M,5.0,9.0,1.0,MMMMM,MMMMM,0,0,0.000000,0.000000,0.000000,0.000000,-4.0
2,19/08/00,Coventry,Middlesbrough,1,3,A,6,16,3,9,8,4,15,21,5,3,1,0,0,0,0,0,0.000000,0.000000,M,M,M,M,M,M,M,M,M,M,14.0,12.0,1.0,MMMMM,MMMMM,0,0,0.000000,0.000000,0.000000,0.000000,2.0
3,19/08/00,Derby,Southampton,2,2,D,6,13,4,6,5,8,11,13,1,1,0,0,0,0,0,0,0.000000,0.000000,M,M,M,M,M,M,M,M,M,M,16.0,15.0,1.0,MMMMM,MMMMM,0,0,0.000000,0.000000,0.000000,0.000000,1.0
4,19/08/00,Leeds,Everton,2,0,H,17,12,8,6,6,4,21,20,1,3,0,0,0,0,0,0,0.000000,0.000000,M,M,M,M,M,M,M,M,M,M,3.0,13.0,1.0,MMMMM,MMMMM,0,0,0.000000,0.000000,0.000000,0.000000,-10.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7595,26/07/2020,Leicester,Manchester United,0,2,A,14,7,3,3,3,3,12,11,1,4,1,0,67,64,39,36,1.631579,1.657895,L,D,W,W,L,D,D,W,W,W,9.0,6.0,38.0,LWLDW,DWDWW,7,11,0.736842,0.736842,-0.026316,-0.105263,3.0
7596,26/07/2020,Manchester City,Norwich,5,0,H,31,5,10,4,9,0,7,4,1,1,0,0,97,26,35,70,2.052632,0.552632,W,L,W,L,W,L,W,L,L,L,1.0,0.0,38.0,WWWWL,LLLLL,12,0,1.631579,-1.157895,1.500000,0.315789,1.0
7597,26/07/2020,Newcastle,Liverpool,1,3,A,3,14,2,6,2,4,11,5,1,0,0,0,37,82,55,32,1.157895,2.526316,D,W,L,L,L,D,L,W,D,W,13.0,2.0,38.0,DLLLD,WLDWW,2,10,-0.473684,1.315789,-1.368421,-0.210526,11.0
7598,26/07/2020,Southampton,Sheffield United,3,1,H,13,5,4,3,9,1,9,16,0,1,0,0,48,38,59,36,1.289474,1.421053,W,L,D,L,D,W,D,W,W,D,16.0,0.0,38.0,WDDDW,LLWWD,9,7,-0.289474,0.052632,-0.131579,0.052632,16.0


In [ ]:
# apply change data in dfp
for x in range (0,len(dfp)):
  dfp['Date'][x]=change_data(dfp,x,'Date')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
# Separate target
resultado = dfp['FTR']

In [ ]:
# apply dummies in results
resultado_d = []
for y in resultado:
  dummies(resultado_d,y)

In [ ]:
# Create Features NoticiasHomeTeam and NoticiasAway
noticias_home = []
noticias_away = []
for x in range(0,len(resultado_d)):
  noticias_home.append('ok')
  noticias_away.append('ok')

In [ ]:
# Create DataFrame with news and results of games 
dataframer ={'Date':dfp['Date'],'HomeTeam':dfp['HomeTeam'],'AwayTeam':dfp['AwayTeam'],'Resultado':resultado_d,'Noticias Home':noticias_home,'Noticias Away':noticias_away}
dataframerpd = pd.DataFrame(dataframer)
dataframerpd.drop_duplicates(inplace = True)
dataframerpd

,Date,HomeTeam,AwayTeam,Resultado,Noticias Home,Noticias Away
0,2000-08-19,Charlton,Manchester City,1,ok,ok
1,2000-08-19,Chelsea,West Ham,1,ok,ok
2,2000-08-19,Coventry,Middlesbrough,3,ok,ok
3,2000-08-19,Derby,Southampton,2,ok,ok
4,2000-08-19,Leeds,Everton,1,ok,ok
...,...,...,...,...,...,...
7595,2020-07-26,Leicester,Manchester United,3,ok,ok
7596,2020-07-26,Manchester City,Norwich,1,ok,ok
7597,2020-07-26,Newcastle,Liverpool,3,ok,ok
7598,2020-07-26,Southampton,Sheffield United,1,ok,ok


In [ ]:
# Create dataframes news of teams
team0=pd.read_csv(path + 'Arsenal FC.csv' )
team1=pd.read_csv(path + 'Aston Villa.csv' )
team2=pd.read_csv(path + 'Brighton &amp; Hove Albion.csv' )
team3=pd.read_csv(path + 'Burnley FC.csv' )
team4=pd.read_csv(path + 'Chelsea FC.csv' )
team5=pd.read_csv(path + 'Crystal Palace.csv' )
team6=pd.read_csv(path + 'Everton FC.csv' )
team7=pd.read_csv(path + 'Fulham FC.csv' )
team8=pd.read_csv(path + 'Leeds United.csv' )
team9=pd.read_csv(path + 'Leicester City.csv' )
team10=pd.read_csv(path + 'Liverpool FC.csv' )
team11=pd.read_csv(path + 'Manchester City.csv' )
team12=pd.read_csv(path + 'Manchester United.csv' )
team13=pd.read_csv(path + 'Newcastle United.csv' )
team14=pd.read_csv(path + 'Sheffield United.csv' )
team15=pd.read_csv(path + 'Southampton FC.csv' )
team16=pd.read_csv(path + 'Tottenham Hotspur.csv' )
team17=pd.read_csv(path + 'West Bromwich Albion.csv' )
team18=pd.read_csv(path + 'West Ham United.csv' )
team19=pd.read_csv(path + 'Wolverhampton Wanderers.csv' )

In [ ]:
# Create diconary relating teams with yours dataframes 
times = {'Arsenal FC':team0,'Aston Villa':team1,'Brighton &amp; Hove Albion':team2,'Burnley FC':team3,'Chelsea FC':team4, 'Crystal Palace':team5,'Everton FC':team6,'Fulham FC':team7,'Leeds United':team8,'Leicester City':team9,'Liverpool FC':team10,'Manchester City':team11,'Manchester United':team12,'Newcastle United':team13,'Sheffield United':team14,'Southampton FC':team15,'Tottenham Hotspur':team16,'West Bromwich Albion':team17,'West Ham United':team18,'Wolverhampton Wanderers':team19}
times_vetor = list(times.keys())

In [ ]:
# Create a Dataframe with games as from 2013
df = dataframerpd[4758:]
df.reset_index(drop = True, inplace= True)
df

,Date,HomeTeam,AwayTeam,Resultado,Noticias Home,Noticias Away
0,2013-01-01,Manchester City,Stoke,1,ok,ok
1,2013-01-01,Southampton,Arsenal,2,ok,ok
2,2013-01-01,Swansea,Aston Villa,2,ok,ok
3,2013-01-01,Tottenham,Reading,1,ok,ok
4,2013-01-01,West Brom,Fulham,3,ok,ok
...,...,...,...,...,...,...
2837,2020-07-26,Leicester,Manchester United,3,ok,ok
2838,2020-07-26,Manchester City,Norwich,1,ok,ok
2839,2020-07-26,Newcastle,Liverpool,3,ok,ok
2840,2020-07-26,Southampton,Sheffield United,1,ok,ok


In [ ]:
# apply data change in news
for x in times_vetor:
  time = times[x]
  for y in range(0,len(time)):
    time['date'][y]=change_data(time,y,'date')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [ ]:
#Add HomeTeam and AwayTeam news
for x in range (0,len(df)):
    df['Noticias Home'][x]=get_news(df,x,times_vetor,'HomeTeam')
    df['Noticias Away'][x]=get_news(df,x,times_vetor,'AwayTeam') 

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exec(code_obj, self.user_global_ns, self.user_ns)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.htm

# Create a fuction to generate a vector that will be use in TfidfVectorizer

In [ ]:
def vetor_lemma(df):
# Create a vetor with news to use in TfidfVectorizer  
  vetor_news = []
  for x in range(0,len(df)):
     vetor_news.append(df['Noticias Home'][x])
     vetor_news.append(df['Noticias Away'][x])
     vector_news_clean = []
  # Clean text of vetor_news
  clean= ['0','1','2','3','4','5','6','7','8','9','@','©','&','/','\"','-']
  for x in range (0,len(vetor_news)):
      add = ''
      if vetor_news[x] is not np.nan:
          for y in range (0,len(vetor_news[x])):
              if(vetor_news[x][y] not in clean):
                  add = add + vetor_news[x][y]
      vector_news_clean.append(add)
  # Drop duplicates in vector_news_clean
  vetor = list(dict.fromkeys(vector_news_clean))
  vector_train =[]
  # Apply lemmatization
  for x in range (0,len(vetor)):
      vector_train.append(" ".join([lemma(wd) for wd in vetor[x].split()]))
  return vector_train

# Create and apply models with First approach to create dictionary with TfidfVectorizer

## Transform news in lists with only one text


In [ ]:
df['Noticias Away'][2837]

array([['European football and survival up for grabs in Premier League finaleThe Premier League will finally reach the end of its longest season on Sunday with plenty still on the line at both ends of the table.Manchester United and Leicester face each other in a shootout for a place in the Champions League next season with Chelsea also hoping to secure their place in the top four. Wolves and Tottenham are vying for one place in the Europa League. And at the bottom it could not be closer with one goal separating Aston Villa just outside the relegation zone from Watford in the bottom three, while Bournemouth also have a chance of beating the drop. Leicester fell out of the top four for the first time since September in midweek after a run of three wins in 13 games either side of the coronavirus shutdown. However, United\'s and Chelsea\'s failures to win on Wednesday means the door is still open for Brendan Rodgers\'s men to salvage all the good work they did early in the season with vic

In [ ]:
# Transform the vectos of HomeTeam in a list with one text
for x in range(0,len(df)):
  textos = ' '
  for y in range(0,len(df['Noticias Home'][x])):
    atual= df['Noticias Home'][x][y][0]
    textos = textos +''+atual
  df['Noticias Home'][x] = textos

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
# Transform the vectos of AwayTeam in a list with one text
for x in range(0,len(df)):
  textos = ' '
  for y in range(0,len(df['Noticias Away'][x])):
    atual= df['Noticias Away'][x][y][0]
    textos = textos +''+atual
  df['Noticias Away'][x] = textos

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
df

,Date,HomeTeam,AwayTeam,Resultado,Noticias Home,Noticias Away
0,2013-01-01,Manchester City,Stoke,1,,
1,2013-01-01,Southampton,Arsenal,2,,
2,2013-01-01,Swansea,Aston Villa,2,,
3,2013-01-01,Tottenham,Reading,1,,
4,2013-01-01,West Brom,Fulham,3,,
...,...,...,...,...,...,...
2837,2020-07-26,Leicester,Manchester United,3,,European football and survival up for grabs i...
2838,2020-07-26,Manchester City,Norwich,1,Arteta has 'faith' in Arsenal board despite s...,
2839,2020-07-26,Newcastle,Liverpool,3,,
2840,2020-07-26,Southampton,Sheffield United,1,,"Wolves bolster Europa League bid, Blades blun..."


In [ ]:
df['Noticias Away'][2837]

' European football and survival up for grabs in Premier League finaleThe Premier League will finally reach the end of its longest season on Sunday with plenty still on the line at both ends of the table.Manchester United and Leicester face each other in a shootout for a place in the Champions League next season with Chelsea also hoping to secure their place in the top four. Wolves and Tottenham are vying for one place in the Europa League. And at the bottom it could not be closer with one goal separating Aston Villa just outside the relegation zone from Watford in the bottom three, while Bournemouth also have a chance of beating the drop. Leicester fell out of the top four for the first time since September in midweek after a run of three wins in 13 games either side of the coronavirus shutdown. However, United\'s and Chelsea\'s failures to win on Wednesday means the door is still open for Brendan Rodgers\'s men to salvage all the good work they did early in the season with victory on

In [ ]:
df_target = df['Resultado']

## Divide data in train and test and create vector to apply in TfidfVectorizer

In [ ]:
# Divide data in training and test
X_train, X_test, y_train, y_test = train_test_split(df,df_target, test_size=0.3, random_state=0,stratify= df_target)

In [ ]:
X_train.reset_index(drop = True,inplace = True)
y_train.reset_index(drop = True,inplace = True)
X_test.reset_index(drop = True,inplace = True)
y_test.reset_index(drop = True,inplace = True)

### Create a vector to use in train

In [ ]:
vector_train = vetor_lemma(X_train)

## Apply TfidfVectorizer

### Vectorizer for training 

In [ ]:
# Fit vectorizer for train
vectorizer_train = TfidfVectorizer(stop_words={'english'},max_features=2000)
vectorizer_train.fit(vector_train)

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=1.0, max_features=2000,
                min_df=1, ngram_range=(1, 1), norm='l2', preprocessor=None,
                smooth_idf=True, stop_words={'english'}, strip_accents=None,
                sublinear_tf=False, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, use_idf=True, vocabulary=None)

In [ ]:
print(vectorizer_train.get_feature_names())
len(vectorizer_train.get_feature_names())

['aaron', 'ability', 'able', 'about', 'above', 'absence', 'absent', 'absolutely', 'abu', 'abuse', 'academy', 'accept', 'accord', 'account', 'accuse', 'achieve', 'achievement', 'acknowledge', 'acros', 'act', 'action', 'adam', 'adamant', 'adapt', 'add', 'added', 'adebayor', 'admit', 'adnan', 'adrift', 'advance', 'advantage', 'affect', 'afford', 'afp', 'african', 'after', 'afternoon', 'again', 'against', 'agbonlahor', 'age', 'agent', 'aggregate', 'ago', 'agree', 'aguero', 'ahead', 'aim', 'ajax', 'alan', 'albion', 'aleksandar', 'alex', 'alexi', 'alive', 'all', 'allardyce', 'allege', 'allen', 'allow', 'alltime', 'almost', 'along', 'alongside', 'already', 'also', 'although', 'alvaro', 'alway', 'amaze', 'ambition', 'amid', 'among', 'amount', 'an', 'analyse', 'and', 'ander', 'anderlecht', 'andre', 'andrea', 'andy', 'anelka', 'anfield', 'angel', 'angle', 'angry', 'ankle', 'anniversary', 'announce', 'another', 'answer', 'anthony', 'antonio', 'any', 'anyone', 'anythe', 'apart', 'apologise', 'appe

2000

In [ ]:
# Transform training news in vectors
tf_noticias_home_train = vectorizer_train.transform(X_train['Noticias Home'])
tf_noticias_away_train = vectorizer_train.transform(X_train['Noticias Away'])

In [ ]:
# Transforms vectors to dense form
tfnoticiash_train = tf_noticias_home_train.todense()
tfnoticiasa_train = tf_noticias_away_train.todense()
X_train_ok = np.hstack((tfnoticiash_train,tfnoticiasa_train))
y_train_ok = np.array(y_train)

### Vectorizer for testing

In [ ]:
# Fit vectorizer for test
vectorizer_test = TfidfVectorizer(stop_words={'english'},max_features=2000)
vectorizer_test.fit(vector_train)

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=1.0, max_features=2000,
                min_df=1, ngram_range=(1, 1), norm='l2', preprocessor=None,
                smooth_idf=True, stop_words={'english'}, strip_accents=None,
                sublinear_tf=False, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, use_idf=True, vocabulary=None)

In [ ]:
print(vectorizer_test.get_feature_names())
len(vectorizer_test.get_feature_names())

['aaron', 'ability', 'able', 'about', 'above', 'absence', 'absent', 'absolutely', 'abu', 'abuse', 'academy', 'accept', 'accord', 'account', 'accuse', 'achieve', 'achievement', 'acknowledge', 'acros', 'act', 'action', 'adam', 'adamant', 'adapt', 'add', 'added', 'adebayor', 'admit', 'adnan', 'adrift', 'advance', 'advantage', 'affect', 'afford', 'afp', 'african', 'after', 'afternoon', 'again', 'against', 'agbonlahor', 'age', 'agent', 'aggregate', 'ago', 'agree', 'aguero', 'ahead', 'aim', 'ajax', 'alan', 'albion', 'aleksandar', 'alex', 'alexi', 'alive', 'all', 'allardyce', 'allege', 'allen', 'allow', 'alltime', 'almost', 'along', 'alongside', 'already', 'also', 'although', 'alvaro', 'alway', 'amaze', 'ambition', 'amid', 'among', 'amount', 'an', 'analyse', 'and', 'ander', 'anderlecht', 'andre', 'andrea', 'andy', 'anelka', 'anfield', 'angel', 'angle', 'angry', 'ankle', 'anniversary', 'announce', 'another', 'answer', 'anthony', 'antonio', 'any', 'anyone', 'anythe', 'apart', 'apologise', 'appe

2000

In [ ]:
# Transform test news in vectors
tf_noticias_home_test = vectorizer_train.transform(X_test['Noticias Home'])
tf_noticias_away_test = vectorizer_train.transform(X_test['Noticias Away'])

In [ ]:
# Transforms vectors to dense form
tfnoticiash_test = tf_noticias_home_test.todense()
tfnoticiasa_test = tf_noticias_away_test.todense()
X_test_ok = np.hstack((tfnoticiash_test,tfnoticiasa_test))
y_test_ok = np.array(y_test)

## Logistic Regression

### Use Cross Validation to find best parameters

In [ ]:
log_reg = LogisticRegression(random_state = 42, max_iter = 1000, multi_class = 'ovr')

# defining the search space
space = dict()
space['solver'] = ['lbfgs', 'newton-cg', 'liblinear']
space['penalty'] = ['l1', 'l2', 'elasticnet']
space['C'] = np.arange(10**-4, 20, 0.01)

# training the model and fitting our data to it
rscv = RandomizedSearchCV(log_reg, space, cv = 3, n_jobs = -1, n_iter = 100,verbose=2)
result = rscv.fit(X_train_ok, y_train_ok)

Fitting 3 folds for each of 100 candidates, totalling 300 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed:   17.1s
[Parallel(n_jobs=-1)]: Done 158 tasks      | elapsed:   55.5s
[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed:  1.9min finished


In [ ]:
#best C and regularization
C = rscv.best_params_
c = C['C']
l_penalty = C['penalty']
l_solver = C['solver']

#printing c (best hyperparameter) and penalty
print("C: ", c, "\nPenalty: ", l_penalty)
print("Solver: ", l_solver)

C:  11.4301 
Penalty:  l2
Solver:  newton-cg


### Traning Logistic Regression Model

In [ ]:
reg_log= LogisticRegression(solver= l_solver,random_state=0,penalty= l_penalty,C= c)
reg_log.fit(X_train_ok,y_train_ok)

LogisticRegression(C=11.4301, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=0, solver='newton-cg', tol=0.0001, verbose=0,
                   warm_start=False)

### Testing Logistic Regression Model

In [ ]:
predictions = reg_log.predict(X_test_ok)

In [ ]:
print(confusion_matrix(y_test_ok,predictions))  
print(classification_report(y_test_ok,predictions))  
print(accuracy_score(y_test_ok, predictions))

[[360   6  25]
 [185   2  15]
 [214   3  43]]
              precision    recall  f1-score   support

           1       0.47      0.92      0.63       391
           2       0.18      0.01      0.02       202
           3       0.52      0.17      0.25       260

    accuracy                           0.47       853
   macro avg       0.39      0.37      0.30       853
weighted avg       0.42      0.47      0.37       853

0.47479484173505276


# Using historic and news in Logistic Regression

## Concatenate historic with news

In [ ]:
# Function to apply dummies in FTR
def dummies_dfpt(d,x,y):
  if d[x][y] == 'H':
    d[x][y]=1
  elif d[x][y]=='D':
    d[x][y]=2
  elif d[x][y]=='A':
    d[x][y]=3

In [ ]:
# Using data as from 2013
dfpt = dfp[4758:]
dfpt.reset_index(drop=True,inplace=True)
for y in range(0,len(dfpt)):
    dummies_dfpt(dfpt,'FTR',y)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: Se

In [ ]:
dfpt

,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HS,AS,HST,AST,HC,AC,HF,AF,HY,AY,HR,AR,HTGS,ATGS,HTGC,ATGC,HTP,ATP,HM1,AM1,HM2,AM2,HM3,AM3,HM4,AM4,HM5,AM5,HomeTeamLP,AwayTeamLP,MW,HTFormPtsStr,ATFormPtsStr,HTFormPts,ATFormPts,HTGD,ATGD,DiffPts,DiffFormPts,DiffLP
0,2013-01-01,Manchester City,Stoke,3,0,1,17,4,9,3,7,2,6,9,2,2,0,0,34,18,16,14,1.950000,1.400000,L,W,W,D,W,D,L,D,D,W,1.0,14.0,20.0,LWWLD,WDDDW,7,9,0.900000,0.200000,0.550000,-0.100000,-13.0
1,2013-01-01,Southampton,Arsenal,1,1,2,9,6,6,4,3,3,9,6,2,0,0,0,26,39,37,21,0.850000,1.650000,D,W,D,W,L,W,W,W,L,L,0.0,3.0,20.0,DDLWL,WWWWL,5,12,-0.550000,0.900000,-0.800000,-0.350000,-3.0
2,2013-01-01,Swansea,Aston Villa,2,2,2,23,8,13,6,3,2,7,15,0,4,0,0,29,15,24,39,1.333333,0.857143,W,L,D,L,D,L,L,W,L,D,11.0,16.0,21.0,WDDLL,LLLWD,5,4,0.238095,-1.142857,0.476190,0.047619,-5.0
3,2013-01-01,Tottenham,Reading,3,1,1,30,7,20,4,12,3,9,9,0,0,0,0,36,22,26,37,1.714286,0.619048,W,W,W,D,D,L,W,L,L,L,4.0,0.0,21.0,WWDWL,WDLLL,10,4,0.476190,-0.714286,1.095238,0.285714,4.0
4,2013-01-01,West Brom,Fulham,1,2,3,17,10,10,5,10,5,14,6,0,0,0,0,28,30,25,36,1.571429,1.000000,L,L,W,D,W,L,D,L,L,W,10.0,9.0,21.0,LWWDL,LDLLW,7,4,0.142857,-0.285714,0.571429,0.142857,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2837,2020-07-26,Leicester,Manchester United,0,2,3,14,7,3,3,3,3,12,11,1,4,1,0,67,64,39,36,1.631579,1.657895,L,D,W,W,L,D,D,W,W,W,9.0,6.0,38.0,LWLDW,DWDWW,7,11,0.736842,0.736842,-0.026316,-0.105263,3.0
2838,2020-07-26,Manchester City,Norwich,5,0,1,31,5,10,4,9,0,7,4,1,1,0,0,97,26,35,70,2.052632,0.552632,W,L,W,L,W,L,W,L,L,L,1.0,0.0,38.0,WWWWL,LLLLL,12,0,1.631579,-1.157895,1.500000,0.315789,1.0
2839,2020-07-26,Newcastle,Liverpool,1,3,3,3,14,2,6,2,4,11,5,1,0,0,0,37,82,55,32,1.157895,2.526316,D,W,L,L,L,D,L,W,D,W,13.0,2.0,38.0,DLLLD,WLDWW,2,10,-0.473684,1.315789,-1.368421,-0.210526,11.0
2840,2020-07-26,Southampton,Sheffield United,3,1,1,13,5,4,3,9,1,9,16,0,1,0,0,48,38,59,36,1.289474,1.421053,W,L,D,L,D,W,D,W,W,D,16.0,0.0,38.0,WDDDW,LLWWD,9,7,-0.289474,0.052632,-0.131579,0.052632,16.0


In [ ]:
# Add NoticiasHome and NoticiasAway for dfpt
dfpt['Noticias Home']=df['Noticias Home']
dfpt['Noticias Away']= df['Noticias Away']

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
# Drop games whitout news
for x in range(0,len(dfpt)):
  if dfpt['Noticias Home'][x]==' ' and dfpt['Noticias Away'][x]==' ':
      dfpt.drop(index=x,inplace= True)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [ ]:
dfpt.reset_index(drop = True, inplace= True)

In [ ]:
# Function to apply Dummies in "HM"and "AM" features
def m_dummies(x,y,w):
    if x[y][w] == 'W':
       x[y][w]=1
    elif x[y][w]=='D':
       x[y][w]=2
    elif x[y][w]=='L':
      x[y][w]=3
    elif x[y][w]=='M':
      x[y][w]=4

In [ ]:
#Apply Dummies in "HM"and "AM" features
new_dummies= ['HM1', 'AM1', 'HM2','AM2', 'HM3', 'AM3', 'HM4', 'AM4', 'HM5', 'AM5']
for x in new_dummies:
  for y in range(0,len(dfpt)):
     m_dummies(dfpt,x,y)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/usr/local/lib

In [ ]:
# Features to use in training
columns = ['HS', 'AS', 'HST', 'AST', 'HC', 'AC', 'HF', 'AF', 'HY', 'AY', 'HR', 'AR', 'HTGS', 'ATGS', 'HTGC', 'ATGC', 'HTP', 'ATP', 'HM1', 'AM1', 'HM2',
       'AM2', 'HM3', 'AM3', 'HM4', 'AM4', 'HM5', 'AM5', 'HomeTeamLP', 'AwayTeamLP','HTGD', 'ATGD', 'DiffPts',
       'DiffFormPts', 'DiffLP','Noticias Home','Noticias Away']


In [ ]:
dfpt

,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HS,AS,HST,AST,HC,AC,HF,AF,HY,AY,HR,AR,HTGS,ATGS,HTGC,ATGC,HTP,ATP,HM1,AM1,HM2,AM2,HM3,AM3,HM4,AM4,HM5,AM5,HomeTeamLP,AwayTeamLP,MW,HTFormPtsStr,ATFormPtsStr,HTFormPts,ATFormPts,HTGD,ATGD,DiffPts,DiffFormPts,DiffLP,Noticias Home,Noticias Away
0,2013-08-17,Arsenal,Aston Villa,1,3,3,16,9,4,4,4,3,15,18,4,5,1,0,0,0,0,0,0.000000,0.000000,4,4,4,4,4,4,4,4,4,4,4.0,15.0,1.0,MMMMM,MMMMM,0,0,0.000000,0.000000,0.000000,0.000000,-11.0,,Football: Bent joins Fulham on loan from Vill...
1,2013-08-17,Swansea,Manchester United,1,4,3,17,15,6,7,7,4,13,10,1,3,0,0,0,0,0,0,0.000000,0.000000,4,4,4,4,4,4,4,4,4,4,9.0,1.0,1.0,MMMMM,MMMMM,0,0,0.000000,0.000000,0.000000,0.000000,8.0,,Football: Rooney returns as champs United hea...
2,2013-08-18,Crystal Palace,Tottenham,0,1,3,5,17,3,2,3,7,6,9,1,0,0,0,0,0,0,0,0.000000,0.000000,4,4,4,4,4,4,4,4,4,4,0.0,5.0,1.0,MMMMM,MMMMM,0,0,0.000000,0.000000,0.000000,0.000000,-5.0,Football: Novelty abounds as Premier League k...,
3,2013-08-19,Manchester City,Newcastle,4,0,1,20,5,11,1,8,1,9,7,2,3,0,1,0,0,0,0,0.000000,0.000000,4,4,4,4,4,4,4,4,4,4,2.0,16.0,1.0,MMMMM,MMMMM,0,0,0.000000,0.000000,0.000000,0.000000,-14.0,Football: Pellegrini faces striking dilemma i...,
4,2013-08-21,Chelsea,Aston Villa,2,1,1,15,7,3,3,1,2,12,13,1,4,0,0,2,3,0,1,1.500000,1.500000,1,1,4,4,4,4,4,4,4,4,3.0,15.0,2.0,WMMMM,WMMMM,3,3,1.000000,1.000000,0.000000,0.000000,-12.0,,Football: Liverpool complete Cissokho loan si...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
765,2020-07-22,Manchester United,West Ham,1,1,2,11,12,4,3,2,3,10,9,3,1,0,0,63,47,35,60,1.675676,1.000000,1,1,2,1,1,3,1,2,1,1,6.0,10.0,37.0,WDWWW,WWLDW,13,10,0.756757,-0.351351,0.675676,0.081081,-4.0,Solskjaer defends 'mentally strong' De Gea af...,
766,2020-07-26,Crystal Palace,Tottenham,1,1,2,13,7,2,2,7,2,11,13,3,2,0,0,30,60,49,46,1.105263,1.526316,3,1,3,1,3,1,3,2,3,1,12.0,4.0,38.0,LLLLL,WWWDW,0,13,-0.500000,0.368421,-0.421053,-0.342105,8.0,"Rashford, Martial and VAR keep Man Utd on cou...",
767,2020-07-26,Leicester,Manchester United,0,2,3,14,7,3,3,3,3,12,11,1,4,1,0,67,64,39,36,1.631579,1.657895,3,2,1,1,3,2,2,1,1,1,9.0,6.0,38.0,LWLDW,DWDWW,7,11,0.736842,0.736842,-0.026316,-0.105263,3.0,,European football and survival up for grabs i...
768,2020-07-26,Manchester City,Norwich,5,0,1,31,5,10,4,9,0,7,4,1,1,0,0,97,26,35,70,2.052632,0.552632,1,3,1,3,1,3,1,3,3,3,1.0,0.0,38.0,WWWWL,LLLLL,12,0,1.631579,-1.157895,1.500000,0.315789,1.0,Arteta has 'faith' in Arsenal board despite s...,


## Divide train and test

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(dfpt[columns],dfpt['FTR'], test_size=0.3, random_state=0,stratify=dfpt['FTR'])

In [ ]:
X_train.reset_index(drop = True,inplace = True)
y_train.reset_index(drop = True,inplace = True)
X_test.reset_index(drop = True,inplace = True)
y_test.reset_index(drop = True,inplace = True)

## Create vectors for Tfidf for train and test

In [ ]:
news_train = vetor_lemma(X_train)

### Vectorizer

#### Vectorizer in train news

In [ ]:
# Fit vectorizer for train
vectorizer_train = TfidfVectorizer(stop_words={'english'},max_features=2000)
vectorizer_train.fit(news_train)

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=1.0, max_features=2000,
                min_df=1, ngram_range=(1, 1), norm='l2', preprocessor=None,
                smooth_idf=True, stop_words={'english'}, strip_accents=None,
                sublinear_tf=False, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, use_idf=True, vocabulary=None)

In [ ]:
train_home = vectorizer_train.transform(X_train['Noticias Home'])
train_away = vectorizer_train.transform(X_train['Noticias Away'])

In [ ]:
# Transforms vectors to dense form
train_home_d = train_home.todense()
train_away_d = train_away.todense()
X_train_ok = np.hstack((train_home_d,train_away_d))
y_train_ok = np.array(y_train)

#### Vectorizer in test news

In [ ]:
test_home = vectorizer_train.transform(X_test['Noticias Home'])
test_away = vectorizer_train.transform(X_test['Noticias Away'])

In [ ]:
# Transforms vectors to dense form
test_home_d = test_home.todense()
test_away_d = test_away.todense()
X_test_ok = np.hstack((test_home_d,test_away_d))
y_test_ok = np.array(y_train)

### Add news in format of vectores in training 

In [ ]:
# Create dataframe with matrix of news
dftrain = pd.DataFrame(X_train_ok)

In [ ]:
# Drop news in format of words
X_train.drop(columns=['Noticias Home','Noticias Away'],inplace=True)

In [ ]:
X_train =pd.concat((X_train,dftrain),ignore_index=True,axis=1)

In [ ]:
columns_news = ['HS', 'AS', 'HST', 'AST', 'HC', 'AC', 'HF', 'AF', 'HY', 'AY', 'HR', 'AR', 'HTGS', 'ATGS', 'HTGC', 'ATGC', 'HTP', 'ATP', 'HM1', 'AM1', 'HM2',
       'AM2', 'HM3', 'AM3', 'HM4', 'AM4', 'HM5', 'AM5', 'HomeTeamLP', 'AwayTeamLP','HTGD', 'ATGD', 'DiffPts',
       'DiffFormPts', 'DiffLP']

In [ ]:
# back name for columns of historic
y = 0
for x in columns_news:
    X_train.rename(columns={y:x},inplace=True)
    y=y+1

In [ ]:
X_train

,HS,AS,HST,AST,HC,AC,HF,AF,HY,AY,HR,AR,HTGS,ATGS,HTGC,ATGC,HTP,ATP,HM1,AM1,HM2,AM2,HM3,AM3,HM4,AM4,HM5,AM5,HomeTeamLP,AwayTeamLP,HTGD,ATGD,DiffPts,DiffFormPts,DiffLP,35,36,37,38,39,...,3995,3996,3997,3998,3999,4000,4001,4002,4003,4004,4005,4006,4007,4008,4009,4010,4011,4012,4013,4014,4015,4016,4017,4018,4019,4020,4021,4022,4023,4024,4025,4026,4027,4028,4029,4030,4031,4032,4033,4034
0,14,16,6,5,5,7,7,17,3,2,0,0,15,13,9,8,1.777778,1.555556,1,2,1,2,3,1,3,3,2,3,10.0,5.0,0.666667,0.555556,0.222222,0.222222,5.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.000000,0.000000,0.107544,0.004449,0.006925,0.0,0.0,0.0,0.0,0.0,0.005327,0.019468,0.0,0.003348,0.0,0.0,0.014104,0.003001,0.005386,0.010663,0.002533,0.00326,0.000000,0.014431,0.0,0.006581,0.003566,0.00341,0.003901,0.00000,0.043065,0.012581,0.0,0.000000,0.000000,0.000000,0.000000,0.019349,0.000000
1,10,12,4,7,7,5,11,8,2,2,0,0,17,11,23,27,1.055556,0.722222,3,3,3,3,3,1,1,1,2,3,15.0,0.0,-0.333333,-0.888889,0.333333,-0.111111,15.0,0.005533,0.010819,0.000000,0.013974,0.015803,...,0.0,0.015939,0.018429,0.092144,0.004720,0.003673,0.0,0.0,0.0,0.0,0.0,0.009418,0.012391,0.0,0.003551,0.0,0.0,0.024935,0.000000,0.002857,0.000000,0.002687,0.00000,0.003243,0.013608,0.0,0.002327,0.000000,0.00000,0.008276,0.00000,0.013536,0.006673,0.0,0.000000,0.000000,0.012222,0.014886,0.000000,0.002975
2,14,19,4,5,6,6,12,16,1,2,0,1,31,43,37,37,1.137931,1.517241,2,1,3,1,3,1,2,2,3,3,10.0,6.0,-0.206897,0.206897,-0.379310,-0.275862,4.0,0.000000,0.000000,0.000000,0.000000,0.065977,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.0,0.000000,0.000000,0.00000,0.000000,0.00000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,12,7,6,2,7,4,15,14,2,3,0,0,6,1,3,2,1.000000,1.000000,3,1,1,3,4,4,4,4,4,4,2.0,0.0,1.000000,-0.333333,0.000000,0.000000,2.0,0.006200,0.003031,0.006779,0.018790,0.000000,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.0,0.000000,0.000000,0.00000,0.000000,0.00000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,12,18,1,7,3,9,12,11,3,3,1,0,4,11,14,13,1.000000,1.100000,3,3,3,3,3,1,3,2,3,3,15.0,6.0,-1.000000,-0.200000,-0.100000,-0.400000,9.0,0.000000,0.000000,0.000000,0.010838,0.045963,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.0,0.000000,0.000000,0.00000,0.000000,0.00000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
534,10,7,4,4,6,2,12,10,0,3,0,0,27,20,37,39,1.000000,1.000000,3,1,2,3,3,2,3,3,1,2,15.0,11.0,-0.357143,-0.678571,0.000000,-0.035714,4.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.0,0.000000,0.000000,0.00000,0.000000,0.00000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
535,22,9,7,4,5,4,7,8,1,0,0,0,34,17,12,16,1.692308,1.153846,1,1,3,2,1,3,3,2,1,1,2.0,9.0,1.692308,0.076923,0.538462,0.076923,-7.0,0.003564,0.006968,0.000000,0.018901,0.008906,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.0,0.000000,0.000000,0.00000,0.000000,0.0000

### Add news in format of vectores is test

In [ ]:
# Create dataframe with matrix of news
dftest = pd.DataFrame(X_test_ok)

In [ ]:
# Drop news in format of words
X_test.drop(columns=['Noticias Home','Noticias Away'],inplace=True)

In [ ]:
X_test =pd.concat((X_test,dftest),ignore_index=True,axis=1)

In [ ]:
# back name for columns of historic
y = 0
for x in columns_news:
    X_test.rename(columns={y:x},inplace=True)
    y=y+1

In [ ]:
X_test

,HS,AS,HST,AST,HC,AC,HF,AF,HY,AY,HR,AR,HTGS,ATGS,HTGC,ATGC,HTP,ATP,HM1,AM1,HM2,AM2,HM3,AM3,HM4,AM4,HM5,AM5,HomeTeamLP,AwayTeamLP,HTGD,ATGD,DiffPts,DiffFormPts,DiffLP,35,36,37,38,39,...,3995,3996,3997,3998,3999,4000,4001,4002,4003,4004,4005,4006,4007,4008,4009,4010,4011,4012,4013,4014,4015,4016,4017,4018,4019,4020,4021,4022,4023,4024,4025,4026,4027,4028,4029,4030,4031,4032,4033,4034
0,11,19,6,9,2,7,13,14,4,3,0,0,26,19,17,19,1.750000,1.312500,1,2,1,1,1,1,1,3,1,3,7.0,2.0,0.562500,0.000000,0.437500,0.500000,5.0,0.006814,0.000000,0.004967,0.030976,0.004865,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000
1,28,3,7,0,12,1,7,9,2,2,0,0,100,40,37,49,2.184211,1.052632,1,1,1,3,1,3,1,3,2,3,2.0,10.0,1.657895,-0.236842,1.131579,0.263158,-8.0,0.001416,0.000000,0.007223,0.018591,0.004043,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000
2,14,22,6,4,4,11,9,9,4,4,0,0,24,65,43,37,0.771429,1.885714,1,2,3,2,3,1,2,1,3,3,0.0,4.0,-0.542857,0.800000,-1.114286,-0.114286,-4.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.005738,0.000000,0.078978,0.000000,0.022479,0.0,0.0,0.0,0.0,0.0,0.004068,0.005947,0.000000,0.000000,0.002433,0.000000,0.020346,0.002292,0.008227,0.000000,0.001935,0.0,0.004670,0.028167,0.0,0.015078,0.0,0.005208,0.008938,0.0,0.034109,0.004804,0.002637,0.001926,0.0,0.000000,0.000000,0.008867,0.000000
3,3,11,2,6,11,3,11,14,1,3,1,0,12,28,34,33,0.846154,1.269231,3,3,3,2,3,1,3,1,3,3,15.0,9.0,-0.846154,-0.192308,-0.423077,-0.269231,6.0,0.000000,0.013216,0.000000,0.020485,0.000000,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000
4,15,12,9,4,2,2,9,13,1,2,0,0,66,45,20,23,2.269231,1.923077,1,1,3,3,1,3,1,1,1,2,1.0,5.0,1.769231,0.846154,0.346154,0.192308,-4.0,0.000000,0.006910,0.012880,0.017850,0.005046,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
226,5,17,1,5,3,7,8,11,2,3,0,0,35,49,47,29,1.040000,1.960000,3,1,3,1,2,2,3,3,2,1,16.0,4.0,-0.480000,0.800000,-0.920000,-0.320000,12.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.039031,0.000000,0.104376,0.000000,0.007710,0.0,0.0,0.0,0.0,0.0,0.005271,0.000000,0.004342,0.000000,0.000000,0.005976,0.013957,0.000000,0.000000,0.010552,0.011281,0.0,0.009076,0.014281,0.0,0.022793,0.0,0.010122,0.011580,0.0,0.030778,0.009337,0.005125,0.011232,0.0,0.000000,0.000000,0.000000,0.004162
227,22,9,5,2,5,4,5,10,1,3,0,0,6,2,2,2,2.000000,1.000000,1,1,1,3,4,4,4,4,4,4,4.0,7.0,1.333333,0.000000,1.000000,1.000000,-3.0,0.000000,0.003471,0.005176,0.034074,0.002535,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0

## Applying Logistic Regression

### Use Cross Validation to find best parameters

In [ ]:
y_trainu =y_train.astype('int')

In [ ]:
log_reg = LogisticRegression(random_state = 42, max_iter = 1000, multi_class = 'ovr')

# defining the search space
space = dict()
space['solver'] = ['lbfgs','newton-cg', 'liblinear']
space['penalty'] = ['l1', 'l2', 'elasticnet']
space['C'] = np.arange(10**-4, 20, 0.01)

# training the model and fitting our data to it
rscv = RandomizedSearchCV(log_reg, space, cv = 3, n_jobs = -1, n_iter = 100,verbose=2)
result = rscv.fit(X_train,y_trainu)

Fitting 3 folds for each of 100 candidates, totalling 300 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed:   59.7s
[Parallel(n_jobs=-1)]: Done 158 tasks      | elapsed:  2.6min
[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed:  5.3min finished


In [ ]:
#best C and regularization
C = rscv.best_params_
c = C['C']
l_penalty = C['penalty']
l_solver = C['solver']

#printing c (best hyperparameter) and penalty
print("C: ", c, "\nPenalty: ", l_penalty)
print("Solver: ", l_solver)

C:  2.0401000000000002 
Penalty:  l1
Solver:  liblinear


### Training model

In [ ]:
reg_log= LogisticRegression(solver=l_solver,random_state=0,penalty= l_penalty,C=c )
reg_log.fit(X_train,y_trainu)

LogisticRegression(C=2.0401000000000002, class_weight=None, dual=False,
                   fit_intercept=True, intercept_scaling=1, l1_ratio=None,
                   max_iter=100, multi_class='auto', n_jobs=None, penalty='l1',
                   random_state=0, solver='liblinear', tol=0.0001, verbose=0,
                   warm_start=False)

### Testing model

In [ ]:
y_testu=y_test.astype('int')

In [ ]:
predictions = reg_log.predict(X_test)
predictions2 = reg_log.predict(X_train)

In [ ]:
print(confusion_matrix(y_testu,predictions))  
print(classification_report(y_testu,predictions))  
print(accuracy_score(y_testu, predictions))
print(accuracy_score(y_trainu, predictions2))

[[85  6 14]
 [23  4 21]
 [14  8 56]]
              precision    recall  f1-score   support

           1       0.70      0.81      0.75       105
           2       0.22      0.08      0.12        48
           3       0.62      0.72      0.66        78

    accuracy                           0.63       231
   macro avg       0.51      0.54      0.51       231
weighted avg       0.57      0.63      0.59       231

0.6277056277056277
0.6957328385899815


## Using logistic regression whitout news

In [ ]:
X_train_wn = X_train[columns_news]

In [ ]:
X_test_wn = X_test[columns_news]

In [ ]:
log_reg2 = LogisticRegression(random_state = 42, max_iter = 1000, multi_class = 'ovr')

# defining the search space
space = dict()
space['solver'] = ['newton-cg', 'liblinear']
space['penalty'] = ['l1', 'l2', 'elasticnet']
space['C'] = np.arange(10**-4, 20, 0.01)

# training the model and fitting our data to it
rscv2 = RandomizedSearchCV(log_reg2, space, cv = 3, n_jobs = -1, n_iter = 100,verbose=2)
result2 = rscv2.fit(X_train_wn, y_trainu)

Fitting 3 folds for each of 100 candidates, totalling 300 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done 220 tasks      | elapsed:    7.0s
[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed:   11.1s finished


In [ ]:
#best C and regularization
C2 = rscv2.best_params_
c2 = C2['C']
l_penalty2 = C2['penalty']
l_solver2 = C2['solver']

#printing c (best hyperparameter) and penalty
print("C: ", c2, "\nPenalty: ", l_penalty2)
print("Solver: ", l_solver2)

C:  2.3001000000000005 
Penalty:  l2
Solver:  liblinear


In [ ]:
reg_log2= LogisticRegression(solver=l_solver2,random_state=0,penalty=l_penalty2,C=c2 )
reg_log2.fit(X_train_wn,y_trainu)

LogisticRegression(C=2.3001000000000005, class_weight=None, dual=False,
                   fit_intercept=True, intercept_scaling=1, l1_ratio=None,
                   max_iter=100, multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=0, solver='liblinear', tol=0.0001, verbose=0,
                   warm_start=False)

In [ ]:
predictions2 = reg_log2.predict(X_test_wn)

In [ ]:
print(confusion_matrix(y_testu,predictions2))  
print(classification_report(y_testu,predictions2))  
print(accuracy_score(y_testu, predictions2))

[[87  7 11]
 [26  2 20]
 [15  8 55]]
              precision    recall  f1-score   support

           1       0.68      0.83      0.75       105
           2       0.12      0.04      0.06        48
           3       0.64      0.71      0.67        78

    accuracy                           0.62       231
   macro avg       0.48      0.53      0.49       231
weighted avg       0.55      0.62      0.58       231

0.6233766233766234
